## This is a notebook that explores the file which is the result of the bash script, that merges trips and leavetimes with one another

In [1]:
import pandas as pd
import os, sys


#reading in data
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, 'bus_lines/bus_66_merge.csv')


df = pd.read_csv(newPath,  delimiter=',', index_col=False)


In [2]:
df.head(5)

,LineID,Direction,RouteID,DayOfService,TripID,ProgrNumber,StopPointID,PlannedTime_Arr,PlannedTime_Dep,ActualTime_Arr,ActualTime_Dep
0,66,1,66_11,2017-01-03,4084361,33,2218,39411,39411,38673,38673
1,66,1,66_11,2017-01-03,4084368,30,2216,69044,69044,68235,68235
2,66,1,66_11,2017-01-03,4084370,33,2218,32185,32185,31840,31840
3,66,1,66_11,2017-01-03,4084398,36,3366,80601,80601,80693,80693
4,66,1,66_11,2017-01-03,4084400,58,3918,44530,44530,44452,44452


In [13]:
df.sort_values(['TripID', 'DayOfService', 'ProgrNumber'], ascending=[True, True,True])

,LineID,Direction,RouteID,DayOfService,TripID,ProgrNumber,StopPointID,PlannedTime_Arr,PlannedTime_Dep,ActualTime_Arr,ActualTime_Dep
181276,66,1,66_16,2017-01-03,4084264,24,2215,65870,65870,65054,65054
181277,66,1,66_16,2017-01-03,4084264,29,2219,66160,66160,65354,65364
181278,66,1,66_16,2017-01-03,4084264,34,3887,66429,66429,65631,65631
181279,66,1,66_16,2017-01-03,4084264,39,3955,66721,66721,65886,65886
181280,66,1,66_16,2017-01-03,4084264,44,3960,66994,66994,66108,66108
181281,66,1,66_16,2017-01-03,4084264,49,3965,67268,67268,66350,66350
181282,66,1,66_16,2017-01-03,4084264,54,5114,67891,67891,66809,66809
181283,66,1,66_16,2017-01-10,4084264,1,346,63600,63600,63850,63850
181284,66,1,66_16,2017-01-10,4084264,2,317,63774,63774,63850,63961
181285,66,1,66_16,2017-01-10,4084264,3,312,64143,64143,64132,64186
